In [1]:
import numpy as np
import pickle
import imageio

import torch
from diffusers import AnimateDiffPipeline, DDIMScheduler, MotionAdapter
from diffusers.utils import export_to_gif, export_to_video
from diffusers.loaders.lora_pipeline import StableDiffusionLoraLoaderMixin

from cdfvd import fvd


d:\IPTP\3A\P2\CSC_52002\lab\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lora_list = ["zoom-in", "zoom-out", "pan-left", "pan-right", "tilt-up", "tilt-down", "rolling-clockwise", "rolling-anticlockwise"]
prompt_list = ["masterpiece, best quality, highly detailed, ultra detailed, cherry blossom trees, petals falling, gentle breeze, spring season, tranquil park, golden sunlight, peaceful atmosphere, vibrant colors",
               "futuristic city, flying cars, neon glow, holograms, sci-fi, cyberpunk, high-tech buildings, rainy streets, reflections, ultra-detailed, cinematic lighting, masterpiece",
               "majestic snow-covered mountains, crystal-clear lake, morning mist, golden sunrise, serene landscape, pine trees, masterpiece, best quality, highly detailed, ultra realistic",
               "ancient temple ruins, overgrown vines, dense jungle, mystical atmosphere, glowing fireflies, moonlight filtering through trees, fantasy landscape, best quality, ultra detailed",
               "a cozy wooden cabin, deep in the snowy forest, warm light from windows, smoke rising from chimney, peaceful winter night, starry sky, soft snowflakes falling, high quality, ultra detailed",
               "deep underwater scene, glowing jellyfish, coral reef, vibrant marine life, sunlight filtering through the ocean, masterpiece, ultra realistic, high detail",
               "medieval castle, on top of a cliff, dramatic sky, storm approaching, lightning in the distance, cinematic atmosphere, ultra detailed, realistic, best quality",
               "a dragon soaring above a burning city, glowing red eyes, smoke and embers in the air, dramatic lighting, fantasy setting, ultra detailed, masterpiece",
               "a futuristic racing scene, high-speed hovercars, neon trails, cybernetic city backdrop, intense motion blur, ultra realistic, high detail, cinematic lighting",
               "a lone samurai standing under a cherry blossom tree, katana in hand, petals drifting in the wind, golden sunset, historical Japan, ultra detailed, masterpiece",
               "desert landscape, ancient ruins, hot sun, sand dunes, a lone traveler walking, long shadows, masterpiece, ultra detailed, cinematic composition",
               "a spaceship landing on an alien planet, strange glowing plants, massive moons in the sky, futuristic technology, sci-fi adventure, ultra realistic, high quality",
               "a giant whale swimming in the clouds, fantasy world, floating islands, warm golden sunlight, peaceful and dreamlike atmosphere, ultra detailed, masterpiece",
               "a Viking ship sailing through stormy seas, thunder and lightning, massive waves crashing, determined warriors on board, cinematic action, ultra realistic, high detail",
               "a neon-lit alleyway in a cyberpunk city, rain pouring down, reflections on the wet pavement, mysterious figure in a trench coat, atmospheric and moody, ultra detailed, cinematic",
               "a tranquil bamboo forest, soft morning light, mist rolling through, peaceful and zen-like atmosphere, ultra realistic, high detail, masterpiece",
               "an astronaut walking on Mars, vast red desert, distant mountains, dramatic lighting, cinematic science fiction, ultra realistic, masterpiece",
               "a mystical forest at twilight, bioluminescent plants, floating orbs of light, magical and otherworldly atmosphere, ultra detailed, best quality",
               "a luxury yacht sailing in a crystal blue ocean, clear sunny sky, gentle waves, people relaxing on deck, ultra realistic, high detail, cinematic",
               "a street festival at night, colorful lanterns, live performances, fireworks in the sky, people celebrating, ultra detailed, masterpiece, lively and vibrant atmosphere"]

In [ ]:
combined_lora_list = ['zoom-in-pan-left',
 'zoom-in-pan-right',
 'zoom-in-tilt-up',
 'zoom-in-tilt-down',
 'zoom-in-rolling-clockwise',
 'zoom-in-rolling-anticlockwise',
 'zoom-out-pan-left',
 'zoom-out-pan-right',
 'zoom-out-tilt-up',
 'zoom-out-tilt-down',
 'zoom-out-rolling-clockwise',
 'zoom-out-rolling-anticlockwise']

In [ ]:
# (
#                 "masterpiece, bestquality, highlydetailed, ultradetailed, sunset, "
#                 "orange sky, warm lighting, fishing boats, ocean waves seagulls, "
#                 "rippling water, wharf, silhouette, serene atmosphere, dusk, evening glow, "
#                 "golden hour, coastal landscape, seaside scenery"
#             )

In [ ]:
def lora_layer_combine(lora_list):
    combined_lora_list = []
    for lora_name_main in lora_list[:2]:
        dict1 = StableDiffusionLoraLoaderMixin.lora_state_dict(f"guoyww/animatediff-motion-lora-{lora_name_main}", cache_dir="./model/lora")[0]
        for lora_name_sub in lora_list[2:]:
            dict2 = StableDiffusionLoraLoaderMixin.lora_state_dict(f"guoyww/animatediff-motion-lora-{lora_name_sub}", cache_dir="./model/lora")[0]
            keys = list(dict1.keys())[:32] + list(dict1.keys())[-48:]
            for key in keys:
                dict1[key] = dict2[key]
            new_name = lora_name_main + "-" + lora_name_sub
            StableDiffusionLoraLoaderMixin.save_lora_weights("model/lora/block0", dict1, weight_name = new_name + ".safetensors")
            combined_lora_list.append(new_name)
    return combined_lora_list



In [8]:
combined_lora_list = lora_layer_combine(lora_list)

d:\IPTP\3A\P2\CSC_52002\lab\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\IPTP\3A\P2\CSC_52002\lab\our_project\model\lora\models--guoyww--animatediff-motion-lora-zoom-in. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\IPTP\3A\P2\CSC_52002\lab\.venv\lib\site-packages\huggingface_hub\file_download.p

In [16]:
adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16, cache_dir="./model/domain_adapter").to("cuda")
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=adapter, torch_dtype=torch.float16, cache_dir="./model/image_motion").to("cuda")
pipe.load_lora_weights("model/lora/block0/zoom-in-pan-left.safetensors", adapter_name = "zoom-in-pan-left")
scheduler = DDIMScheduler.from_pretrained(
        model_id,
        subfolder="scheduler",
        clip_sample=False,
        beta_schedule="linear",
        timestep_spacing="linspace",
        steps_offset=1,)
pipe.scheduler = scheduler
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()
test2 = pipe(
            height = 256,
            width = 256,
            prompt = (
                "masterpiece, bestquality, highlydetailed, ultradetailed, sunset, "
                "orange sky, warm lighting, fishing boats, ocean waves seagulls, "
                "rippling water, wharf, silhouette, serene atmosphere, dusk, evening glow, "
                "golden hour, coastal landscape, seaside scenery"
            ),
            negative_prompt="bad quality, worse quality",
            num_frames=16,
            guidance_scale=7.5,
            num_inference_steps=40,
            output_type = "pil",)
test2 = test2.frames[0]

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.
100%|██████████| 40/40 [00:23<00:00,  1.67it/s]


In [13]:
export_to_video(test2, "test4.mp4")

'test4.mp4'

In [ ]:
def motion_simple(lora_name : str, prompt : str, sample_num :int = 1, device = "cuda"):
    adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16, cache_dir="./model/domain_adapter").to(device)
    model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
    pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=adapter, torch_dtype=torch.float16, cache_dir="./model/image_motion").to(device)
    pipe.load_lora_weights("guoyww/animatediff-motion-lora-" + lora_name, adapter_name=lora_name, cache_dir="./model/lora") 
    scheduler = DDIMScheduler.from_pretrained(
            model_id,
            subfolder="scheduler",
            clip_sample=False,
            beta_schedule="linear",
            timestep_spacing="linspace",
            steps_offset=1,)
    pipe.scheduler = scheduler
    pipe.enable_vae_slicing()
    pipe.enable_model_cpu_offload()
    output = []
    for num in range(sample_num):
        output_unit = pipe(
            height = 256,
            width = 256,
            prompt = prompt,
            negative_prompt="bad quality, worse quality",
            num_frames=16,
            guidance_scale=7.5,
            num_inference_steps=30,
            output_type = "pil",)
        #'np', 'pt', 'pil'
        output.append(output_unit.frames[0])
    # for i in range(sample_num):
    #     np.save(f"data/generated/{lora_name}/prompt_1_array_{i}.npy", output[i])
    for i in range(sample_num):
        export_to_video(output[i], f"data/generated/{lora_name}/prompt_1_video_{i}.mp4")
    return output_unit

In [3]:
def motion_baseline_generate(lora_list, prompt_list, samples_per_lora_per_prompt, device = "cuda"):
    model_id = "SG161222/Realistic_Vision_V5.1_noVAE"    
    adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16, cache_dir="./model/domain_adapter").to(device)
    scheduler = DDIMScheduler.from_pretrained(
            model_id,
            subfolder="scheduler",
            clip_sample=False,
            beta_schedule="linear",
            timestep_spacing="linspace",
            steps_offset=1,)
    pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=adapter, scheduler = scheduler, torch_dtype=torch.float16, cache_dir="./model/image_motion").to(device)
    output = {}
    for lora_name in lora_list:
        output[lora_name] = {}
        pipe.load_lora_weights("guoyww/animatediff-motion-lora-" + lora_name, adapter_name=lora_name, cache_dir="./model/lora")
        pipe.enable_vae_slicing()
        pipe.enable_model_cpu_offload()
        for idx,prompt in enumerate(prompt_list):
            output[lora_name][idx] = []
            for i in range(samples_per_lora_per_prompt):
                output_unit = pipe(
                    height = 256,
                    width = 256,
                    prompt = prompt,
                    negative_prompt="bad quality, worse quality",
                    num_frames=16,
                    guidance_scale=7.5,
                    num_inference_steps=40,
                    output_type = "pil",)
                output_content = output_unit.frames[0]
                export_to_video(output_content, f"data/generated/{lora_name}/prompt_{idx}_video_{i}.mp4")
                output[lora_name][idx].append(output_content)
    return output
                


In [ ]:
baseline = motion_baseline_generate(["pan-right","tilt-up","tilt-down"], prompt_list, 10)

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.
Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  2.90it/s]
d:\IPTP\3A\P2\CSC_52002\lab\.venv\lib\site-packages\transformers\models\clip\modeling_clip.py:539: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  0%|          | 0/40 [00:00<?, ?it/s]

: 

In [3]:
test1 = motion_simple("zoom-out",sample_num = 1)

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.
Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  2.92it/s]
d:\IPTP\3A\P2\CSC_52002\lab\.venv\lib\site-packages\transformers\models\clip\modeling_clip.py:539: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 25/25 [01:03<00:00,  2.52s/it]


In [4]:
test1

AnimateDiffPipelineOutput(frames=[[<PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606560>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606C20>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606590>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606DD0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606AD0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606CB0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606B60>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606200>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF6062F0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606530>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606650>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606680>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606AA0>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606E00>, <PIL.Image.Image image mode=RGB size=512x512 at 0x201DF606E30>, <PIL.

In [12]:
test1.frames.shape

(1, 16, 512, 512, 3)

In [32]:
test1["frames"].shape

(1, 16, 512, 512, 3)

In [2]:
evaluator = fvd.cdfvd('videomae', ckpt_path="model/cdfvd_evaluator/videomae.pth", n_fake = 'full')

Loading videomae model ...


d:\IPTP\3A\P2\CSC_52002\lab\.venv\lib\site-packages\cdfvd\third_party\VideoMAEv2\utils.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_

In [3]:
evaluator.load_videos('sky', data_type='stats_pkl', resolution=256, sequence_length=16)

Loaded 1966 features from d:\IPTP\3A\P2\CSC_52002\lab\.venv\lib\site-packages\cdfvd\fvd_stats_cache\videomae_sky_full_res256_len16_skip1_seed0.pkl


In [ ]:
evaluator.compute_fake_stats(evaluator.load_videos(video_info = "data/generated/mp4", data_type = "video_folder"))

Loading from video files ...


100%|██████████| 1/1 [00:06<00:00,  6.38s/it]


In [7]:
score = evaluator.compute_fvd_from_stats()

In [8]:
score

1536.9371605297827

In [30]:
values[0].shape

(1, 16, 512, 512, 3)

In [33]:
vid_arr = test1["frames"].squeeze(0)

In [40]:
vid_arr = (vid_arr * 255).astype(np.uint8)

In [41]:
vid_list = list(vid_arr)

In [42]:
vid_list[0].shape

(512, 512, 3)

In [45]:
imageio.mimsave("animations/test4.gif", vid_list, fps =10, loop = 0)

In [ ]:
def motion_combine_baseline(lora_names : list | tuple, weights = None, device = "cuda"):
    adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16, cache_dir="./model/domain_adapter").to(device)
    model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
    pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=adapter, torch_dtype=torch.float16, cache_dir="./model/image_motion").to(device)
    for name in lora_names:
        pipe.load_lora_weights("guoyww/animatediff-motion-lora-" + name, adapter_name=name, cache_dir="./model/lora")

    if not weights:
        pipe.set_adapters(lora_names, adapter_weights=[1.0, 1.0])


In [ ]:
def motion_controller(lora_names, block_id, ):
